In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.metrics import mean_squared_error
import math

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
train  = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submit = pd.read_csv('Sample_submission.csv')

In [7]:
a = {'Low Fat':1, 'Normal Fat':2, 'Ultra Low fat':0}

train['Product_Fat_Content'] = train['Product_Fat_Content'].map(a)
test['Product_Fat_Content'] = test['Product_Fat_Content'].map(a)

In [8]:
train['Supermarket _Size'] = train['Supermarket _Size'].fillna('High')

In [9]:
test['Supermarket _Size'] = test['Supermarket _Size'].fillna('High')

In [10]:
b = {'High':2, 'Medium':1, 'Small':0}

train['Supermarket _Size'] = train['Supermarket _Size'].map(b)
test['Supermarket _Size'] = test['Supermarket _Size'].map(b)

In [11]:
c = {'Cluster 3':2, 'Cluster 2':1, 'Cluster 1':0}

train['Supermarket_Location_Type'] = train['Supermarket_Location_Type'].map(c)
test['Supermarket_Location_Type'] = test['Supermarket_Location_Type'].map(c)

In [12]:
d = {'Supermarket Type1':0, 'Supermarket Type2':1, 'Supermarket Type3':2, 'Grocery Store':3}

train['Supermarket_Type'] = train['Supermarket_Type'].map(d)
test['Supermarket_Type'] = test['Supermarket_Type'].map(d)

In [13]:
e = train['Product_Weight'].mean()

train['Product_Weight'] = train['Product_Weight'].fillna(e)

In [14]:
f = test['Product_Weight'].mean()

test['Product_Weight'] = test['Product_Weight'].fillna(f)

In [15]:
g = {'Fruits and Vegetables':0, 'Snack Foods':1, 'Household':2, 'Frozen Foods':3, 'Canned':4, 'Baking Goods':5, 'Dairy':6, 'Health and Hygiene':7, 'Meat':8, 'Soft Drinks':9, 'Hard Drinks':10, 'Breads':11, 'Others':12, 'Starchy Foods':13, 'Breakfast':14, 'Seafood':15}

train['Product_Type'] = train['Product_Type'].map(g)
test['Product_Type'] = test['Product_Type'].map(g)

In [16]:
train.drop('Product_Supermarket_Identifier', axis=1, inplace=True)
test.drop('Product_Supermarket_Identifier', axis=1, inplace=True)

In [59]:
train['PPW'] = train['Product_Weight'] * train['Product_Price']
test['PPW'] = test['Product_Weight'] * test['Product_Price']

In [62]:
train['APP'] = train['Average_Price_per_ProductType'] / train['Product_Price']
test['APP'] = test['Average_Price_per_ProductType'] / test['Product_Price']

In [105]:
train['PSV'] = train['Product_Shelf_Visibility'] * 100
test['PSV'] = test['Product_Shelf_Visibility'] * 100

In [106]:
w_train = train.drop(['Product_Identifier', 'Supermarket_Identifier'], axis=1)
w_test = test.drop(['Product_Identifier', 'Supermarket_Identifier'], axis=1)

In [107]:
X = w_train.drop('Product_Supermarket_Sales', axis=1)
y = w_train['Product_Supermarket_Sales']

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [75]:
model1 = LinearRegression()

In [76]:
model1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [77]:
pred = model1.predict(X_test)

In [78]:
score = mean_squared_error(y_test, pred)

In [79]:
math.sqrt(score)

3625.9100168308414

In [80]:
submit.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,3836.448242
1,FDY10_CHUKWUDI018,4366.989746
2,FDO40_CHUKWUDI018,5528.634766
3,NCQ43_CHUKWUDI018,4151.886719
4,NCC07_CHUKWUDI046,3707.140869


In [81]:
final_pred = model1.predict(w_test)

In [82]:
final_pred[:10]

array([ 5551.94596708,  4083.51561441,  6066.90055969,  3906.44994923,
        3810.81649898,  1395.21190505, 10036.89112898,  9250.22300547,
        4447.88608896,  7076.99105405])

In [83]:
submit['Product_Supermarket_Sales'] = final_pred

In [84]:
submit.head(10)

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,5551.945967
1,FDY10_CHUKWUDI018,4083.515614
2,FDO40_CHUKWUDI018,6066.900560
3,NCQ43_CHUKWUDI018,3906.449949
4,NCC07_CHUKWUDI046,3810.816499
5,FDK27_CHUKWUDI019,1395.211905
6,DRJ39_CHUKWUDI027,10036.891129
7,FDZ16_CHUKWUDI035,9250.223005
8,FDD21_CHUKWUDI018,4447.886089
9,FDQ12_CHUKWUDI019,7076.991054


In [85]:
submit.to_csv('fourth_submission.csv', index=False)

In [86]:
from catboost import CatBoostRegressor

In [87]:
cat_model = CatBoostRegressor()

In [88]:
cat_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=500, use_best_model=True)

Learning rate set to 0.053687
0:	learn: 4338.0353645	test: 4338.0353645	test1: 4473.3692361	best: 4473.3692361 (0)	total: 3.49ms	remaining: 3.49s
1:	learn: 4224.7646863	test: 4224.7646863	test1: 4364.1938643	best: 4364.1938643 (1)	total: 6.54ms	remaining: 3.27s
2:	learn: 4120.8140823	test: 4120.8140823	test1: 4263.1186050	best: 4263.1186050 (2)	total: 9.92ms	remaining: 3.3s
3:	learn: 4028.2335278	test: 4028.2335278	test1: 4175.9502469	best: 4175.9502469 (3)	total: 13.1ms	remaining: 3.26s
4:	learn: 3937.4192334	test: 3937.4192334	test1: 4085.3350205	best: 4085.3350205 (4)	total: 16.3ms	remaining: 3.24s
5:	learn: 3865.2196955	test: 3865.2196955	test1: 4018.2933517	best: 4018.2933517 (5)	total: 19.4ms	remaining: 3.22s
6:	learn: 3790.2904094	test: 3790.2904094	test1: 3944.4387106	best: 3944.4387106 (6)	total: 22.6ms	remaining: 3.21s
7:	learn: 3717.2735309	test: 3717.2735309	test1: 3874.8616744	best: 3874.8616744 (7)	total: 25.8ms	remaining: 3.2s
8:	learn: 3654.4591810	test: 3654.4591810	te

99:	learn: 2737.6539659	test: 2737.6539659	test1: 3060.7407410	best: 3058.6481130 (93)	total: 319ms	remaining: 2.88s
100:	learn: 2735.5269838	test: 2735.5269838	test1: 3061.3023917	best: 3058.6481130 (93)	total: 323ms	remaining: 2.87s
101:	learn: 2732.6747366	test: 2732.6747366	test1: 3062.9647545	best: 3058.6481130 (93)	total: 326ms	remaining: 2.87s
102:	learn: 2730.9307743	test: 2730.9307743	test1: 3063.4120091	best: 3058.6481130 (93)	total: 330ms	remaining: 2.87s
103:	learn: 2727.6368781	test: 2727.6368781	test1: 3064.1305188	best: 3058.6481130 (93)	total: 333ms	remaining: 2.87s
104:	learn: 2725.9784023	test: 2725.9784023	test1: 3063.8046762	best: 3058.6481130 (93)	total: 336ms	remaining: 2.87s
105:	learn: 2724.8101839	test: 2724.8101839	test1: 3064.3883654	best: 3058.6481130 (93)	total: 340ms	remaining: 2.87s
106:	learn: 2722.8432090	test: 2722.8432090	test1: 3063.9320607	best: 3058.6481130 (93)	total: 343ms	remaining: 2.86s
107:	learn: 2721.4763390	test: 2721.4763390	test1: 3063.1

199:	learn: 2550.3465762	test: 2550.3465762	test1: 3077.1212230	best: 3058.6481130 (93)	total: 645ms	remaining: 2.58s
200:	learn: 2547.1854540	test: 2547.1854540	test1: 3077.4335235	best: 3058.6481130 (93)	total: 648ms	remaining: 2.58s
201:	learn: 2545.8867211	test: 2545.8867211	test1: 3077.6443068	best: 3058.6481130 (93)	total: 651ms	remaining: 2.57s
202:	learn: 2543.3252771	test: 2543.3252771	test1: 3078.1912268	best: 3058.6481130 (93)	total: 655ms	remaining: 2.57s
203:	learn: 2540.8715350	test: 2540.8715350	test1: 3079.2628598	best: 3058.6481130 (93)	total: 658ms	remaining: 2.57s
204:	learn: 2538.5217030	test: 2538.5217030	test1: 3080.1860841	best: 3058.6481130 (93)	total: 661ms	remaining: 2.56s
205:	learn: 2537.8430975	test: 2537.8430975	test1: 3080.4324758	best: 3058.6481130 (93)	total: 665ms	remaining: 2.56s
206:	learn: 2535.9626258	test: 2535.9626258	test1: 3079.4153909	best: 3058.6481130 (93)	total: 668ms	remaining: 2.56s
207:	learn: 2535.2421583	test: 2535.2421583	test1: 3079.

297:	learn: 2387.2653700	test: 2387.2653700	test1: 3098.9638625	best: 3058.6481130 (93)	total: 972ms	remaining: 2.29s
298:	learn: 2385.7794442	test: 2385.7794442	test1: 3099.7050550	best: 3058.6481130 (93)	total: 976ms	remaining: 2.29s
299:	learn: 2384.2390929	test: 2384.2390929	test1: 3099.2047471	best: 3058.6481130 (93)	total: 979ms	remaining: 2.28s
300:	learn: 2382.3232051	test: 2382.3232051	test1: 3100.9202466	best: 3058.6481130 (93)	total: 983ms	remaining: 2.28s
301:	learn: 2380.5477914	test: 2380.5477914	test1: 3102.6791278	best: 3058.6481130 (93)	total: 989ms	remaining: 2.29s
302:	learn: 2377.9609302	test: 2377.9609302	test1: 3102.6485935	best: 3058.6481130 (93)	total: 994ms	remaining: 2.29s
303:	learn: 2376.5732708	test: 2376.5732708	test1: 3102.8894524	best: 3058.6481130 (93)	total: 998ms	remaining: 2.29s
304:	learn: 2374.2295829	test: 2374.2295829	test1: 3101.4530251	best: 3058.6481130 (93)	total: 1s	remaining: 2.28s
305:	learn: 2373.2189576	test: 2373.2189576	test1: 3101.248

397:	learn: 2231.5764779	test: 2231.5764779	test1: 3114.6318553	best: 3058.6481130 (93)	total: 1.3s	remaining: 1.96s
398:	learn: 2230.1497965	test: 2230.1497965	test1: 3115.1202390	best: 3058.6481130 (93)	total: 1.3s	remaining: 1.96s
399:	learn: 2229.4466123	test: 2229.4466123	test1: 3115.7858278	best: 3058.6481130 (93)	total: 1.3s	remaining: 1.96s
400:	learn: 2228.3219226	test: 2228.3219226	test1: 3115.5987502	best: 3058.6481130 (93)	total: 1.31s	remaining: 1.95s
401:	learn: 2227.1039588	test: 2227.1039588	test1: 3115.7386601	best: 3058.6481130 (93)	total: 1.31s	remaining: 1.95s
402:	learn: 2225.7469493	test: 2225.7469493	test1: 3116.0449809	best: 3058.6481130 (93)	total: 1.31s	remaining: 1.95s
403:	learn: 2223.6017580	test: 2223.6017580	test1: 3116.0941001	best: 3058.6481130 (93)	total: 1.32s	remaining: 1.94s
404:	learn: 2221.9924075	test: 2221.9924075	test1: 3116.0525027	best: 3058.6481130 (93)	total: 1.32s	remaining: 1.94s
405:	learn: 2220.1728456	test: 2220.1728456	test1: 3116.312

484:	learn: 2115.8377838	test: 2115.8377838	test1: 3125.3007382	best: 3058.6481130 (93)	total: 1.63s	remaining: 1.73s
485:	learn: 2115.3388772	test: 2115.3388772	test1: 3125.6565413	best: 3058.6481130 (93)	total: 1.63s	remaining: 1.72s
486:	learn: 2114.3876940	test: 2114.3876940	test1: 3125.6528767	best: 3058.6481130 (93)	total: 1.63s	remaining: 1.72s
487:	learn: 2113.8493887	test: 2113.8493887	test1: 3125.5855128	best: 3058.6481130 (93)	total: 1.64s	remaining: 1.72s
488:	learn: 2111.9745644	test: 2111.9745644	test1: 3124.7718744	best: 3058.6481130 (93)	total: 1.64s	remaining: 1.71s
489:	learn: 2111.1710006	test: 2111.1710006	test1: 3125.0161776	best: 3058.6481130 (93)	total: 1.64s	remaining: 1.71s
490:	learn: 2110.4383741	test: 2110.4383741	test1: 3125.1105039	best: 3058.6481130 (93)	total: 1.65s	remaining: 1.71s
491:	learn: 2108.7433788	test: 2108.7433788	test1: 3125.1459033	best: 3058.6481130 (93)	total: 1.65s	remaining: 1.7s
492:	learn: 2107.1881258	test: 2107.1881258	test1: 3125.2

583:	learn: 2007.6401567	test: 2007.6401567	test1: 3141.8528677	best: 3058.6481130 (93)	total: 1.95s	remaining: 1.39s
584:	learn: 2006.2857903	test: 2006.2857903	test1: 3141.8953310	best: 3058.6481130 (93)	total: 1.96s	remaining: 1.39s
585:	learn: 2004.6990426	test: 2004.6990426	test1: 3141.3206987	best: 3058.6481130 (93)	total: 1.96s	remaining: 1.39s
586:	learn: 2004.0657817	test: 2004.0657817	test1: 3141.4759155	best: 3058.6481130 (93)	total: 1.96s	remaining: 1.38s
587:	learn: 2002.5379769	test: 2002.5379769	test1: 3142.0620974	best: 3058.6481130 (93)	total: 1.97s	remaining: 1.38s
588:	learn: 2002.1953426	test: 2002.1953426	test1: 3142.4619347	best: 3058.6481130 (93)	total: 1.97s	remaining: 1.37s
589:	learn: 2000.7599123	test: 2000.7599123	test1: 3141.9232936	best: 3058.6481130 (93)	total: 1.97s	remaining: 1.37s
590:	learn: 2000.1038810	test: 2000.1038810	test1: 3141.9307580	best: 3058.6481130 (93)	total: 1.98s	remaining: 1.37s
591:	learn: 1997.7590729	test: 1997.7590729	test1: 3142.

In [89]:
cat_model.fit(X, y)

Learning rate set to 0.047443
0:	learn: 4383.8588646	total: 4.96ms	remaining: 4.95s
1:	learn: 4282.6913143	total: 8.24ms	remaining: 4.11s
2:	learn: 4187.6715112	total: 11.5ms	remaining: 3.82s
3:	learn: 4098.4849863	total: 14.7ms	remaining: 3.67s
4:	learn: 4014.9697259	total: 17.9ms	remaining: 3.56s
5:	learn: 3935.9834695	total: 21.3ms	remaining: 3.53s
6:	learn: 3866.6310489	total: 24.6ms	remaining: 3.49s
7:	learn: 3797.8037549	total: 27.9ms	remaining: 3.46s
8:	learn: 3737.6057210	total: 31.2ms	remaining: 3.43s
9:	learn: 3678.4385712	total: 34.4ms	remaining: 3.41s
10:	learn: 3622.6145485	total: 37.7ms	remaining: 3.39s
11:	learn: 3572.1275672	total: 41.1ms	remaining: 3.38s
12:	learn: 3523.8999435	total: 43.5ms	remaining: 3.3s
13:	learn: 3476.8474232	total: 46.9ms	remaining: 3.3s
14:	learn: 3437.3643538	total: 50.2ms	remaining: 3.29s
15:	learn: 3396.6927990	total: 53.6ms	remaining: 3.29s
16:	learn: 3358.5497315	total: 56.9ms	remaining: 3.29s
17:	learn: 3325.6601540	total: 60.2ms	remaining

191:	learn: 2691.0830860	total: 661ms	remaining: 2.78s
192:	learn: 2689.5777725	total: 664ms	remaining: 2.78s
193:	learn: 2688.7450970	total: 669ms	remaining: 2.78s
194:	learn: 2687.8691445	total: 672ms	remaining: 2.77s
195:	learn: 2686.3582950	total: 675ms	remaining: 2.77s
196:	learn: 2684.8666454	total: 679ms	remaining: 2.77s
197:	learn: 2683.7135335	total: 683ms	remaining: 2.76s
198:	learn: 2682.7431686	total: 686ms	remaining: 2.76s
199:	learn: 2681.8308044	total: 689ms	remaining: 2.76s
200:	learn: 2680.0697984	total: 692ms	remaining: 2.75s
201:	learn: 2679.1557341	total: 696ms	remaining: 2.75s
202:	learn: 2677.9330657	total: 699ms	remaining: 2.74s
203:	learn: 2676.9543734	total: 703ms	remaining: 2.74s
204:	learn: 2675.1121280	total: 708ms	remaining: 2.75s
205:	learn: 2674.5419070	total: 712ms	remaining: 2.74s
206:	learn: 2672.5632448	total: 716ms	remaining: 2.74s
207:	learn: 2671.4728089	total: 719ms	remaining: 2.74s
208:	learn: 2669.8603651	total: 722ms	remaining: 2.73s
209:	learn

376:	learn: 2443.0729479	total: 1.32s	remaining: 2.19s
377:	learn: 2441.8267461	total: 1.33s	remaining: 2.18s
378:	learn: 2441.5280988	total: 1.33s	remaining: 2.18s
379:	learn: 2439.9972300	total: 1.33s	remaining: 2.17s
380:	learn: 2439.0449740	total: 1.34s	remaining: 2.17s
381:	learn: 2437.8426205	total: 1.34s	remaining: 2.17s
382:	learn: 2436.9433220	total: 1.34s	remaining: 2.17s
383:	learn: 2435.4978391	total: 1.35s	remaining: 2.16s
384:	learn: 2434.1254126	total: 1.35s	remaining: 2.16s
385:	learn: 2433.0387488	total: 1.36s	remaining: 2.16s
386:	learn: 2431.4867376	total: 1.36s	remaining: 2.16s
387:	learn: 2431.0719211	total: 1.36s	remaining: 2.15s
388:	learn: 2428.8181157	total: 1.37s	remaining: 2.15s
389:	learn: 2426.8942571	total: 1.37s	remaining: 2.14s
390:	learn: 2426.6495692	total: 1.37s	remaining: 2.14s
391:	learn: 2424.7157055	total: 1.38s	remaining: 2.14s
392:	learn: 2423.8014282	total: 1.38s	remaining: 2.13s
393:	learn: 2421.9642897	total: 1.38s	remaining: 2.13s
394:	learn

527:	learn: 2282.7537761	total: 1.82s	remaining: 1.63s
528:	learn: 2282.0609379	total: 1.83s	remaining: 1.63s
529:	learn: 2281.2172965	total: 1.83s	remaining: 1.62s
530:	learn: 2280.8028904	total: 1.83s	remaining: 1.62s
531:	learn: 2279.7544003	total: 1.84s	remaining: 1.62s
532:	learn: 2279.3823779	total: 1.84s	remaining: 1.61s
533:	learn: 2279.1347608	total: 1.85s	remaining: 1.61s
534:	learn: 2278.4790890	total: 1.85s	remaining: 1.61s
535:	learn: 2277.2848750	total: 1.85s	remaining: 1.6s
536:	learn: 2276.3451307	total: 1.86s	remaining: 1.6s
537:	learn: 2275.0283183	total: 1.86s	remaining: 1.6s
538:	learn: 2273.8444968	total: 1.86s	remaining: 1.59s
539:	learn: 2272.9056174	total: 1.87s	remaining: 1.59s
540:	learn: 2272.0550532	total: 1.87s	remaining: 1.59s
541:	learn: 2270.4285362	total: 1.87s	remaining: 1.58s
542:	learn: 2269.2118818	total: 1.88s	remaining: 1.58s
543:	learn: 2268.9474948	total: 1.88s	remaining: 1.58s
544:	learn: 2267.8007612	total: 1.88s	remaining: 1.57s
545:	learn: 2

720:	learn: 2112.7221643	total: 2.49s	remaining: 962ms
721:	learn: 2112.3910298	total: 2.49s	remaining: 959ms
722:	learn: 2111.6290219	total: 2.49s	remaining: 955ms
723:	learn: 2110.9096177	total: 2.5s	remaining: 951ms
724:	learn: 2110.2633993	total: 2.5s	remaining: 948ms
725:	learn: 2109.5844320	total: 2.5s	remaining: 944ms
726:	learn: 2108.7318788	total: 2.5s	remaining: 941ms
727:	learn: 2107.2493041	total: 2.51s	remaining: 937ms
728:	learn: 2106.8169485	total: 2.51s	remaining: 935ms
729:	learn: 2105.7325467	total: 2.52s	remaining: 931ms
730:	learn: 2105.0101643	total: 2.52s	remaining: 928ms
731:	learn: 2104.1047848	total: 2.52s	remaining: 924ms
732:	learn: 2103.4815894	total: 2.53s	remaining: 921ms
733:	learn: 2102.5699763	total: 2.53s	remaining: 918ms
734:	learn: 2102.2586475	total: 2.54s	remaining: 915ms
735:	learn: 2101.6692602	total: 2.54s	remaining: 912ms
736:	learn: 2101.4550200	total: 2.54s	remaining: 908ms
737:	learn: 2100.7991204	total: 2.55s	remaining: 905ms
738:	learn: 20

911:	learn: 1975.8166832	total: 3.15s	remaining: 305ms
912:	learn: 1975.2343723	total: 3.16s	remaining: 301ms
913:	learn: 1974.5393957	total: 3.16s	remaining: 298ms
914:	learn: 1973.4552566	total: 3.17s	remaining: 294ms
915:	learn: 1972.7856565	total: 3.17s	remaining: 291ms
916:	learn: 1972.4245650	total: 3.17s	remaining: 287ms
917:	learn: 1972.2254127	total: 3.18s	remaining: 284ms
918:	learn: 1971.6729987	total: 3.18s	remaining: 280ms
919:	learn: 1971.1365054	total: 3.19s	remaining: 277ms
920:	learn: 1970.4112618	total: 3.19s	remaining: 274ms
921:	learn: 1969.6037878	total: 3.19s	remaining: 270ms
922:	learn: 1969.0926067	total: 3.2s	remaining: 267ms
923:	learn: 1968.0794891	total: 3.2s	remaining: 263ms
924:	learn: 1967.1206900	total: 3.2s	remaining: 260ms
925:	learn: 1966.3674277	total: 3.21s	remaining: 256ms
926:	learn: 1965.7391809	total: 3.21s	remaining: 253ms
927:	learn: 1964.6823276	total: 3.21s	remaining: 249ms
928:	learn: 1964.4568902	total: 3.22s	remaining: 246ms
929:	learn: 1

In [90]:
cat_pred = cat_model.predict(w_test)

In [91]:
cat_pred[:10]

array([ 3481.27038471,  4269.99230847,  5028.42463378,  3982.86839025,
        4467.21074387,   672.86139361, 12968.57905035,  6978.83623129,
        3973.51561874,   172.14841693])

In [39]:
submit.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,0
1,FDY10_CHUKWUDI018,0
2,FDO40_CHUKWUDI018,0
3,NCQ43_CHUKWUDI018,0
4,NCC07_CHUKWUDI046,0


In [92]:
submit['Product_Supermarket_Sales'] = cat_pred

In [93]:
submit.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,3481.270385
1,FDY10_CHUKWUDI018,4269.992308
2,FDO40_CHUKWUDI018,5028.424634
3,NCQ43_CHUKWUDI018,3982.868390
4,NCC07_CHUKWUDI046,4467.210744


In [94]:
submit.to_csv('fifth_submission.csv', index=False)

In [95]:
from xgboost import XGBRegressor

In [96]:
xg_model = XGBRegressor()

In [97]:
xg_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=500)

[0]	validation_0-rmse:5689.83447	validation_1-rmse:5895.00000
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 500 rounds.
[1]	validation_0-rmse:4454.71289	validation_1-rmse:4722.04492
[2]	validation_0-rmse:3677.94214	validation_1-rmse:3997.65039
[3]	validation_0-rmse:3203.57153	validation_1-rmse:3573.75122
[4]	validation_0-rmse:2914.12134	validation_1-rmse:3367.70776
[5]	validation_0-rmse:2752.98267	validation_1-rmse:3253.02124
[6]	validation_0-rmse:2636.72485	validation_1-rmse:3207.56104
[7]	validation_0-rmse:2564.51807	validation_1-rmse:3180.89404
[8]	validation_0-rmse:2512.59448	validation_1-rmse:3170.62695
[9]	validation_0-rmse:2460.90186	validation_1-rmse:3165.84888
[10]	validation_0-rmse:2437.46851	validation_1-rmse:3177.61792
[11]	validation_0-rmse:2416.13550	validation_1-rmse:3172.15112
[12]	validation_0-rmse:2352.20996	validation_1-rmse:3184.10474
[13]	validation_0-rmse:2315.596

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [98]:
xg_model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [99]:
xg_pred = xg_model.predict(w_test)

In [100]:
submit.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,3481.270385
1,FDY10_CHUKWUDI018,4269.992308
2,FDO40_CHUKWUDI018,5028.424634
3,NCQ43_CHUKWUDI018,3982.868390
4,NCC07_CHUKWUDI046,4467.210744


In [101]:
submit['Product_Supermarket_Sales'] = xg_pred

In [102]:
submit.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,3273.279297
1,FDY10_CHUKWUDI018,4586.031250
2,FDO40_CHUKWUDI018,5807.517090
3,NCQ43_CHUKWUDI018,3996.299072
4,NCC07_CHUKWUDI046,4099.684570


In [103]:
submit.to_csv('sixth_submission.csv', index=False)

In [112]:
from sklearn.ensemble import RandomForestRegressor

In [114]:
rfc_model = RandomForestRegressor(n_estimators=1000, max_depth=None, min_samples_split=10, max_leaf_nodes=50)

In [117]:
rfc_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=50,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [118]:
rfc_pred = rfc_model.predict(X_test)

In [120]:
rfc_score = mean_squared_error(y_test, rfc_pred)

In [121]:
print(math.sqrt(rfc_score))

3120.771193140186


In [122]:
rfc_final = rfc_model.predict(w_test)

In [123]:
rfc_final[:10]

array([ 3807.14370015,  4536.66179458,  5974.69193582,  4044.95608473,
        4000.77682179,   828.03577209, 15733.91095064,  7444.52933379,
        4468.77648812,   832.20894108])

In [124]:
submit.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,FDY57_CHUKWUDI013,3273.279297
1,FDY10_CHUKWUDI018,4586.031250
2,FDO40_CHUKWUDI018,5807.517090
3,NCQ43_CHUKWUDI018,3996.299072
4,NCC07_CHUKWUDI046,4099.684570


In [125]:
submit['Product_Supermarket_Sales'] = rfc_final

In [126]:
submit.to_csv('rfc_pred.csv', index=None)